In [64]:

from cmath import inf
import matplotlib.pyplot as plt
import numpy as np
import deap
from deap import creator
import pydot 
from IPython.display import Image, display
import networkx as nx
import random
import math


class TrieNode:
 
    def __init__(self, primitive):
        self.primitive = primitive
        self.path = 'root'
        self.traverse_count = 0
        self.total_cv_score = []
        self.generation = []
        self.children = {}
        self.parents = []
        self.depth = 0
        self.max_score = -inf
        self.min_score = inf
        self.diversity_score = 0
 
class DeapTrie(object):
 
    def __init__(self):
        self.root = TrieNode("")
        
    def insert(self, pipeline_str,pipeline_data,pset):

        def prim_to_list(prim, args):
            if isinstance(prim, deap.gp.Terminal):
                return None
            return [prim.name] + args
        def remove_none(obj):
            if isinstance(obj, (list, tuple, set)):
                return type(obj)(remove_none(x) for x in obj if x is not None)
            elif isinstance(obj, dict):
                return type(obj)((remove_none(k), remove_none(v))
                for k, v in obj.items() if k is not None and v is not None)
            else:
                return obj

        pipeline = creator.Individual.from_string(pipeline_str, pset)

        #convert pipeline into a list and change all hyperparameters to None
        tree = []
        stack = []
        for node in pipeline:
            stack.append((node, []))
            while len(stack[-1][1]) == stack[-1][0].arity:
                prim, args = stack.pop()
                tree = prim_to_list(prim, args)
                if len(stack) == 0:
                    break  # If stack is empty, all nodes should have been seen
                
                stack[-1][1].append(tree)
        
        #remove all Nones
        tree = remove_none(tree)
        
        #dfs through the tree and integrate into trie
        stack = []
        stack.append(tree)
        trie_stack = [self.root]

        while stack:
            s = stack.pop()
            node = trie_stack.pop()
            cur_depth = node.depth+1
            
            if (s[0]) not in node.children:
                node.children[(s[0])] = TrieNode(s[0])
                node.children[(s[0])].parents = np.append(node.parents,node)
                #add a value to the root diversity metric
                #self.root.diversity_score =  self.root.diversity_score + 1/cur_depth**2
                temp_depth = 1
                for tempnode in node.parents:
                    tempnode.diversity_score =  tempnode.diversity_score + 1/temp_depth**2
                    temp_depth = temp_depth + 1
            node.children[(s[0])].traverse_count = node.children[(s[0])].traverse_count + 1
            node.children[(s[0])].total_cv_score.append(pipeline_data["internal_cv_score"])
            node.children[(s[0])].generation.append(pipeline_data["generation"])
            node.children[(s[0])].depth = cur_depth
            if not math.isnan(pipeline_data["internal_cv_score"]) and not math.isinf(pipeline_data["internal_cv_score"]):
                node.children[(s[0])].min_score = min(node.children[(s[0])].min_score,pipeline_data["internal_cv_score"])
                node.children[(s[0])].max_score = max(node.children[(s[0])].max_score,pipeline_data["internal_cv_score"])
                self.root.min_score = min(self.root.min_score,pipeline_data["internal_cv_score"])
                self.root.max_score = max(self.root.max_score,pipeline_data["internal_cv_score"])
            if node.path != 'root':
                node.children[(s[0])].path = node.path + '-' + s[0]
            else:
                node.children[(s[0])].path = s[0]
            if len(s[1:]) > 0:
                stack.extend(s[1:])
                for i in range(len(s[1:])):
                    trie_stack.append(node.children[(s[0])])
                    

                
    def display(self,filename, depth=100):
        import networkx as nx
        from pyvis.network import Network
        import matplotlib as mpl

        def colorFader(c1,c2,mix=0): #fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
            if mix < 0:
                mix = 0
            if mix > 1:
                mix = 1
            c1=np.array(mpl.colors.to_rgb(c1))
            c2=np.array(mpl.colors.to_rgb(c2))
            return mpl.colors.to_hex((1-mix)*c1 + mix*c2)

        c1='red' #blue
        c2='green' #green

        graph = pydot.Dot(graph_type='graph') 
        stack = [self.root]
        parent_stack = []

        max_height = depth
        while stack:
            s = stack.pop()
            if s.depth >= max_height:
                continue
            for k in s.children.keys():
                stack.append(s.children[k])
                temp =  [v for v in s.total_cv_score if not math.isnan(v) and not math.isinf(v)]
                if len(temp) :
                    parentnodeaccuracy =(sum(temp)/len(temp))
                    if parentnodeaccuracy > self.root.max_score:
                        parentnodeaccuracy = self.root.max_score
                    parentnodecolor = colorFader(c1,c2,(parentnodeaccuracy-self.root.min_score)/(self.root.max_score-self.root.min_score))
                else:
                    parentnodeaccuracy = 'NA'
                    parentnodecolor = "#666666"
                    
                temp =  [v for v in s.children[k].total_cv_score if not math.isnan(v) and not math.isinf(v)]
                if len(temp) :
                    childaccuracy = (sum(temp)/len(temp))
                    #floating point 0.00...01 issue
                    if childaccuracy > self.root.max_score:
                        childaccuracy = self.root.max_score
                    childcolor = colorFader(c1,c2,(childaccuracy-self.root.min_score)/(self.root.max_score-self.root.min_score))
                    
                else:
                    childaccuracy = 'NA'
                    childcolor = "#666666"
                
                graph.add_node(pydot.Node(s.path,label=s.primitive+'\n'+str(parentnodeaccuracy),color=parentnodecolor,size=10*(math.tanh(-s.depth+4)+2)))
                graph.add_node(pydot.Node(s.children[k].path,label=s.children[k].primitive+'\n'+str(childaccuracy),color=childcolor,size=10*(math.tanh(-s.children[k].depth+4)+2)))
                
                edge = pydot.Edge(s.path, s.children[k].path,weight=1,color='#515ba3',value=math.log(s.children[k].traverse_count))
                graph.add_edge(edge)



        G = nx.nx_pydot.from_pydot(graph)
        nt = Network(bgcolor='#333333', font_color='white', height="100%",width="100%")
        nt.from_nx(G)
        nt.show(filename+'.html')
 


In [16]:
import random
import numpy as np
from deap import base, creator, tools

NUMBER_OF_CITIES = 100
INDIVIDUAL_SIZE = 20
distances = np.zeros((NUMBER_OF_CITIES, NUMBER_OF_CITIES))
for city in range(NUMBER_OF_CITIES):
    cities = [ i for i in range(NUMBER_OF_CITIES) if not i == city ]
    for to_city in cities:
        distances[to_city][city] = \
            distances[city][to_city] = random.randint(50, 2000)

## Min.
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)
toolbox = base.Toolbox()
## permutation setup for individual,
toolbox.register("indices", \
                 random.sample, \
                 range(INDIVIDUAL_SIZE), 
                 INDIVIDUAL_SIZE)
toolbox.register("individual", \
                 tools.initIterate, \
                 creator.Individual, toolbox.indices)
## population setup,
toolbox.register("population", \
                 tools.initRepeat, \
                 list, toolbox.individual)

def EVALUATE(individual):
    summation = 0
    start = individual[0]
    for i in range(1, len(individual)):
        end = individual[i]
        summation += distances[start][end]
        start = end
    return summation
toolbox.register("evaluate", EVALUATE)

toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.01)
toolbox.register("select", tools.selTournament, tournsize=10)

POPULATION_SIZE = 200
N_ITERATIONS = 1000
N_MATINGS = 50
a = Runner(toolbox)
a.set_parameters(POPULATION_SIZE, N_ITERATIONS, N_MATINGS)
stats, population = a.Run()

FileNotFoundError: [Errno 2] No such file or directory: 'tsp/gr17.json'

DEAP-provided TSP problem:

In [53]:

import array
import random
import json

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

# gr*.json contains the distance map in list of list style in JSON format
# Optimal solutions are : gr17 = 2085, gr24 = 1272, gr120 = 6942
with open("../examples/ga/tsp/gr120.json", "r") as tsp_data:
    tsp = json.load(tsp_data)

distance_map = tsp["DistanceMatrix"]
IND_SIZE = tsp["TourSize"]

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", array.array, typecode='i', fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Attribute generator
toolbox.register("indices", random.sample, range(IND_SIZE), IND_SIZE)

# Structure initializers
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evalTSP(individual):
    distance = distance_map[individual[-1]][individual[0]]
    for gene1, gene2 in zip(individual[0:-1], individual[1:]):
        distance += distance_map[gene1][gene2]
    return distance,

toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evalTSP)


random.seed(169)

pop = toolbox.population(n=300)

hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)


pop, logbook = algorithms.eaSimple(pop, toolbox, 0.7, 0.2, 50, stats=stats, 
                    halloffame=hof)
# return pop, stats, hof


/Users/matsumoton/opt/anaconda3/envs/exploration_trie/lib/python3.10/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Users/matsumoton/opt/anaconda3/envs/exploration_trie/lib/python3.10/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	avg    	std    	min  	max  
0  	300   	52224.2	2335.79	45169	58507
1  	234   	51109.4	2288.37	44872	57115
2  	239   	50358.6	2243.21	44939	57306
3  	229   	49786.5	2240.61	45000	56698
4  	249   	49784.1	2531.54	44135	56814
5  	229   	49288  	2574.16	44135	56263
6  	233   	49357  	2719.83	44135	57266
7  	225   	48893.8	2563.32	43775	56048
8  	225   	48600.4	2700.58	43551	56746
9  	221   	48380.8	2639.49	43508	59256
10 	239   	48573.2	2827.56	42105	57479
11 	233   	48746.9	2892.48	42818	56172
12 	231   	48231.1	2837.33	42687	57985
13 	207   	47883  	2825.17	39108	55403
14 	230   	48095.3	2787.6 	39664	56047
15 	216   	48036.5	2768.58	41607	55566
16 	232   	47721.7	2741.76	41607	55313
17 	228   	47730.2	3019.12	41607	56000
18 	226   	47343.6	2837.47	41607	58012
19 	229   	47119.6	2812.45	41607	54941
20 	220   	46862.7	2827.52	41064	57658
21 	224   	46744.4	2741.15	41064	53909
22 	238   	46967.1	3028.33	38941	55951
23 	231   	46896.8	3030.71	38941	55122
24 	216   	45956.1	2891.4

In [67]:
toolbox.__dict__

{'clone': functools.partial(<function deepcopy at 0x7fb450127c70>),
 'map': functools.partial(<class 'map'>),
 'indices': functools.partial(<bound method Random.sample of <random.Random object at 0x7fb46b01d410>>, range(0, 120), 120),
 'individual': functools.partial(<function initIterate at 0x7fb43026c8b0>, <class 'deap.creator.Individual'>, functools.partial(<bound method Random.sample of <random.Random object at 0x7fb46b01d410>>, range(0, 120), 120)),
 'population': functools.partial(<function initRepeat at 0x7fb43026c820>, <class 'list'>, functools.partial(<function initIterate at 0x7fb43026c8b0>, <class 'deap.creator.Individual'>, functools.partial(<bound method Random.sample of <random.Random object at 0x7fb46b01d410>>, range(0, 120), 120))),
 'mate': functools.partial(<function cxPartialyMatched at 0x7fb43024ac20>),
 'mutate': functools.partial(<function mutShuffleIndexes at 0x7fb43026cc10>, indpb=0.05),
 'select': functools.partial(<function selTournament at 0x7fb43026d090>, to

In [94]:
import deap
class TrieNode:
 
    def __init__(self, primitive):
        self.primitive = primitive
        self.path = 'root'
        self.traverse_count = 0
        self.total_cv_score = []
        self.generation = []
        self.children = {}
        self.parents = []
        self.depth = 0
        self.max_score = -inf
        self.min_score = inf
        self.diversity_score = 0
 
class DeapTrie(object):
 
    def __init__(self, individual_type):
        self.root = TrieNode("")
        self.individual_type = individual_type
        
    def insert(self, individual, pset = None):

        def prim_to_list(prim, args):
            if isinstance(prim, deap.gp.Terminal):
                return None
            return [prim.name] + args
        def remove_none(obj):
            if isinstance(obj, (list, tuple, set)):
                return type(obj)(remove_none(x) for x in obj if x is not None)
            elif isinstance(obj, dict):
                return type(obj)((remove_none(k), remove_none(v))
                for k, v in obj.items() if k is not None and v is not None)
            else:
                return obj

        if deap_trie.individual_type == 'array.array':
            individual = individual.tolist()
            tree = individual
        elif deap_trie.individual_type == 'deap.gp.PrimitiveTree':
            individual = creator.Individual.from_string(individual, pset)

            #convert pipeline into a list and change all hyperparameters to None
            tree = []
            stack = []
            for node in individual:
                stack.append((node, []))
                while len(stack[-1][1]) == stack[-1][0].arity:
                    prim, args = stack.pop()
                    tree = prim_to_list(prim, args)
                    if len(stack) == 0:
                        break  # If stack is empty, all nodes should have been seen
                    
                    stack[-1][1].append(tree)
            
            #remove all Nones
            tree = remove_none(tree)
        
        #dfs through the tree and integrate into trie
        stack = []
        stack.append(tree)
        trie_stack = [self.root]

        while stack:
            s = stack.pop()
            node = trie_stack.pop()
            cur_depth = node.depth+1
            
            if (s[0]) not in node.children:
                node.children[(s[0])] = TrieNode(s[0])
                node.children[(s[0])].parents = np.append(node.parents,node)
                #add a value to the root diversity metric
                #self.root.diversity_score =  self.root.diversity_score + 1/cur_depth**2
                temp_depth = 1
                for tempnode in node.parents:
                    tempnode.diversity_score =  tempnode.diversity_score + 1/temp_depth**2
                    temp_depth = temp_depth + 1
            node.children[(s[0])].traverse_count = node.children[(s[0])].traverse_count + 1
            node.children[(s[0])].total_cv_score.append(pipeline_data["internal_cv_score"])
            node.children[(s[0])].generation.append(pipeline_data["generation"])
            node.children[(s[0])].depth = cur_depth
            if not math.isnan(pipeline_data["internal_cv_score"]) and not math.isinf(pipeline_data["internal_cv_score"]):
                node.children[(s[0])].min_score = min(node.children[(s[0])].min_score,pipeline_data["internal_cv_score"])
                node.children[(s[0])].max_score = max(node.children[(s[0])].max_score,pipeline_data["internal_cv_score"])
                self.root.min_score = min(self.root.min_score,pipeline_data["internal_cv_score"])
                self.root.max_score = max(self.root.max_score,pipeline_data["internal_cv_score"])
            if node.path != 'root':
                node.children[(s[0])].path = node.path + '-' + s[0]
            else:
                node.children[(s[0])].path = s[0]
            if len(s[1:]) > 0:
                stack.extend(s[1:])
                for i in range(len(s[1:])):
                    trie_stack.append(node.children[(s[0])])
                    

deap_trie = DeapTrie(array.array)
for individual in pop:
    deap_trie.insert(individual)

UnboundLocalError: local variable 'tree' referenced before assignment